In [1]:
# Optimizer와 경사하강법
# 최적화는 각 학습 단계에서 모델의 오류를 줄이기 위해 모델 매개변수를 조정하는 과정으로 Optimizer는 최적화 알고리즘을 의미한다.
# 대표적인 최적화 알고리즘이 확률적 경사하강법(SGD)
# Pytorch 에는 모델과 데이터 타입에 따라 보다 좋은 성능을 제공하는 ADAM이나 RMSProop과 같은 다양한 옵티마이저가 있다.

In [2]:
# 사용법
# oprimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# 학습단계
# optimizer.zero_grad()를 호출하여 모델 파라미터의 미분값(변화도) 를 0 으로 초기화해준다.   = detach_().requires_grad_(True) 이거랑 같은 듯
# loss.backward()를 호출하여 backward pass를 계산하고 연산에 연결된 각 텐서들의 미분 값을 각 텐서객체 grad에 저장한다.
# optimizer.step()을 호출하여 역전파 단계에서 계산된 미분값(변화도) 를 기준으로 모델 파라미터 값을 업데이트 한다.

In [12]:
import torch
w = torch.tensor(4.0, requires_grad = True)
z = 2 * w
z.backward()
print(w.grad)

z = 2 * w
z.backward()
print(w.grad)

z = 2 * w
z.backward()
print(w.grad)

# 끊어주지 않으니 계속 chain rule 되는 것을 알 수 있음
# 이걸 끊어 줘야 함

tensor(2.)
tensor(4.)
tensor(6.)


In [22]:
import torch
x = torch.ones(4)
y = torch.zeros(3)
w = torch.rand(4,3, requires_grad = True)
b = torch.rand(3, requires_grad = True)

In [23]:
learning_rate = 0.01
optimizer = torch.optim.SGD([w,b], lr = learning_rate)

In [24]:
import torch.nn.functional as F
nb_epochs = 300

for epoch in range(nb_epochs + 1):
    z = torch.matmul(x,w) + b
    loss = F.mse_loss(z, y)

    optimizer.zero_grad() # 초기화
    loss.backward() # 편미분 계산
    optimizer.step() # 업데이트

    if epoch % 100 == 0: # 100 번마다 출력
        print("에폭: ",epoch)
        print("nb_epochs: ",nb_epochs)
        print("weight: ",w)
        print("bias: ",b)
        print("loss: ",loss)
        print("----------------"*5)

에폭:  0
nb_epochs:  300
weight:  tensor([[0.2717, 0.6532, 0.8424],
        [0.7917, 0.3057, 0.4152],
        [0.6449, 0.9604, 0.0752],
        [0.0209, 0.0227, 0.8180]], requires_grad=True)
bias:  tensor([0.5316, 0.9110, 0.8020], requires_grad=True)
loss:  tensor(7.8372, grad_fn=<MseLossBackward0>)
--------------------------------------------------------------------------------
에폭:  100
nb_epochs:  300
weight:  tensor([[-0.1652,  0.1018,  0.2717],
        [ 0.3548, -0.2456, -0.1554],
        [ 0.2080,  0.4090, -0.4955],
        [-0.4161, -0.5287,  0.2473]], requires_grad=True)
bias:  tensor([0.0947, 0.3596, 0.2313], requires_grad=True)
loss:  tensor(0.0089, grad_fn=<MseLossBackward0>)
--------------------------------------------------------------------------------
에폭:  200
nb_epochs:  300
weight:  tensor([[-0.1800,  0.0833,  0.2525],
        [ 0.3401, -0.2642, -0.1747],
        [ 0.1933,  0.3904, -0.5147],
        [-0.4308, -0.5473,  0.2281]], requires_grad=True)
bias:  tensor([0.0800, 

In [26]:
# 모듈을 이용해서 더 간단하게 만들 수 있도록 하기
import torch
import torch.nn as nn
import torch.nn.functional as F

class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.linear = nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        return self.linear(x)
    
model = LinearRegressionModel(4,3)

In [28]:
x = torch.ones(4)
y = torch.zeros(3)

learning_rate = 0.01
nb_epochs = 1000
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

for epoch in range(nb_epochs + 1):
    
    pred = model(x)
    loss = F.mse_loss(pred, y)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [31]:
print(loss)
for param in model.parameters():
    print(param)

tensor(1.2570e-13, grad_fn=<MseLossBackward0>)
Parameter containing:
tensor([[ 0.0377, -0.0468,  0.1864, -0.1111],
        [ 0.4505, -0.2300, -0.3133,  0.0983],
        [-0.3646, -0.4818,  0.3857,  0.3771]], requires_grad=True)
Parameter containing:
tensor([-0.0663, -0.0053,  0.0837], requires_grad=True)


In [33]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_diabetes

diabets_data = load_diabetes()

diabets_data.DESCR

'.. _diabetes_dataset:\n\nDiabetes dataset\n----------------\n\nTen baseline variables, age, sex, body mass index, average blood\npressure, and six blood serum measurements were obtained for each of n =\n442 diabetes patients, as well as the response of interest, a\nquantitative measure of disease progression one year after baseline.\n\n**Data Set Characteristics:**\n\n  :Number of Instances: 442\n\n  :Number of Attributes: First 10 columns are numeric predictive values\n\n  :Target: Column 11 is a quantitative measure of disease progression one year after baseline\n\n  :Attribute Information:\n      - age     age in years\n      - sex\n      - bmi     body mass index\n      - bp      average blood pressure\n      - s1      tc, total serum cholesterol\n      - s2      ldl, low-density lipoproteins\n      - s3      hdl, high-density lipoproteins\n      - s4      tch, total cholesterol / HDL\n      - s5      ltg, possibly log of serum triglycerides level\n      - s6      glu, blood sugar